<a href="https://colab.research.google.com/github/drewamorbordelon/DS-Unit-2-Kaggle-Challenge/blob/master/module3-cross-validation/DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
import pandas as pd
import numpy as np

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv')).set_index('id')

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

#Wrangle Data
Import Data

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'),
        pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv', index_col='id')

In [5]:
train.head(3)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional


#EDA

In [6]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(train, minimal=True).to_notebook_iframe()

# profile

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

#Problems we need to deal with:
High cardinality
Zeroes an NaN
NaNs
Features constant values in columns

In [8]:
def wrangle (X):
  X = X.copy()

  # Latitude - replacing the funky latitude measurements with 0
  X['latitude'] = X['latitude'].replace(-2e-08, 0)

  # Latitude and Longitude 0's to NaN
  for col in ['latitude', 'longitude']:
    X[col] = X[col].replace(0, np.nan)

  # Drop high cardinality columns
  hc_cols = [col for col in X.describe(include='object').columns
             if X[col].nunique() > 100]

  X = X.drop(hc_cols, axis=1) # Actually dropping hc_cols(high cardinality)

  # Drop Columns (`quantity_group` is repeated with `quantity`)
  X = X.drop(['quantity_group', 'recorded_by', 'payment_type', 'num_private', 
              'extraction_type_group', 'extraction_type_class', 'payment', 
              'source'], axis=1)
              
  return X

In [9]:
train = wrangle(train)
test = wrangle(test)
train.head()

,id,amount_tsh,gps_height,longitude,latitude,basin,region,region_code,district_code,population,public_meeting,scheme_management,permit,construction_year,extraction_type,management,management_group,water_quality,quality_group,quantity,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,1390,34.938093,-9.856322,Lake Nyasa,Iringa,11,5,109,True,VWC,False,1999,gravity,vwc,user-group,soft,good,enough,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,1399,34.698766,-2.147466,Lake Victoria,Mara,20,2,280,NaN,Other,True,2010,gravity,wug,user-group,soft,good,insufficient,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,686,37.460664,-3.821329,Pangani,Manyara,21,4,250,True,VWC,True,2009,gravity,vwc,user-group,soft,good,enough,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,263,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,90,63,58,True,VWC,True,1986,submersible,vwc,user-group,soft,good,dry,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,0,31.130847,-1.825359,Lake Victoria,Kagera,18,1,0,True,NaN,True,0,gravity,other,other,soft,good,seasonal,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [10]:
train['construction_year'] = train['construction_year'].replace(0, 1999)

# Target vector feature matrix SPLIT

In [11]:
y = train['status_group']
X = train.drop('status_group', axis=1)

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

#Establish Baseline

In [12]:
print('Baseline Accuracy:', y_train.value_counts(normalize=True).max())

Baseline Accuracy: 0.5429713804713805


#Build Model

In [13]:
# Import Libraries for building a model
from sklearn.pipeline import make_pipeline
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier


model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    StandardScaler(),
    LogisticRegression(n_jobs=-1)
)

model.fit(X, y)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['basin', 'region', 'public_meeting',
                                     'scheme_management', 'permit',
                                     'extraction_type', 'management',
                                     'management_group', 'water_quality',
                                     'quality_group', 'quantity', 'source_type',
                                     'source_class', 'waterpoint_type',
                                     'waterpoint_type_group'],
                               drop_invariant=False, handle_missing='value',
                               handle_unknow...
                               verbose=0)),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept

#Check Metrics

In [14]:
print('Training Accuracy:', model.score(X_train, y_train))
print('Validation Accuracy:', model.score(X_val, y_val))

Training Accuracy: 0.7286405723905723
Validation Accuracy: 0.7296296296296296


#Tune the Model

In [15]:
# Import GridSearchCV (cross-validation)
from sklearn.model_selection import GridSearchCV

rfc_model = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(max_depth=18, n_jobs=-1, random_state=42) # <-- max_depth, n_estimators
)
# The ranges you want to test, as a dictionary
params = {'randomforestclassifier__n_estimators': range(50, 201, 50), # 4 options
          'randomforestclassifier__max_depth': range(5, 26, 10)} # 3 options

# Create your gridsearch
gs = GridSearchCV(rfc_model,
                  param_grid=params,
                  n_jobs=-1,
                  verbose=1,
                  cv=5,
                  scoring='accuracy'
)    

In [ ]:
gs.fit(X_train, y_train);

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
# What are the best set of hyperparameters?
gs.best_params_

In [ ]:
# What if I want to save my best model?
best_model = gs.best_estimator_

In [ ]:
print('Training Accuracy:', best_model.score(X_train, y_train))
print('Validation Accuracy:', best_model.score(X_val, y_val))

In [ ]:
best_model.fit(X_train, y_train)
print('Validation Accuracy', best_model.score(X_val, y_val))
y_pred = best_model.predict(X_val)

In [ ]:
test['prediction'] = best_model.predict(test)
submission = test.filter(['prediction'], axis=1)
submission.columns =['status_group']
submission['id'] = submission.index
submission = submission.rename_axis('index1').reset_index()
submission.drop('index1',axis=1,inplace=True)
submission = submission[['id', 'status_group']]
submission

In [ ]:
submission.to_csv(r'drew-submission-01-rfc.csv', index=False)

from google.colab import files
files.download('drew-submission-01-rfc.csv')